#analise do case de rh base do https://www.kaggle.com/mzinic/employee-churn-prediction?select=employee_data.csv

In [1]:
import pandas as pd
from scipy import stats
import numpy as np

In [2]:
#importando a base

data=pd.read_csv('employee_data.csv')

In [3]:

data['resposta']=np.where(data['status']=='Left',1,0)

data = pd.get_dummies(data, columns = ['department'])
data ['last_evaluation'] = data ['last_evaluation']. fillna (0)
data['new']=np.where(data ['last_evaluation'] ==0,1,0)
data ['filed_complaint'] = data ['filed_complaint']. fillna (0)
data = pd.get_dummies(data, columns = ['salary'])

data ['recently_promoted'] = data ['recently_promoted']. fillna (0)

data ['satisfaction'] = data ['satisfaction']. fillna (data ['satisfaction'].mean())

#tenure                                 
data ['tenure'] = data ['tenure']. fillna (data ['tenure'].mean())


In [4]:
data.isnull().sum()

avg_monthly_hrs                      0
filed_complaint                      0
last_evaluation                      0
n_projects                           0
recently_promoted                    0
satisfaction                         0
status                               0
tenure                               0
resposta                             0
department_IT                        0
department_admin                     0
department_engineering               0
department_finance                   0
department_information_technology    0
department_management                0
department_marketing                 0
department_procurement               0
department_product                   0
department_sales                     0
department_support                   0
department_temp                      0
new                                  0
salary_high                          0
salary_low                           0
salary_medium                        0
dtype: int64

In [5]:

feature_cols = [
'avg_monthly_hrs',
'filed_complaint',
'last_evaluation',
'n_projects',     
'recently_promoted',
'salary_high',           
'salary_low',           
'salary_high',           
'salary_medium',
    'satisfaction',     
'tenure',           
'department_IT',    
'department_admin', 
'department_engineering',
'department_finance',               
'department_information_technology',
'department_management',          
'department_marketing',                
'department_procurement',               
'department_product',                   
'department_sales',                     
'department_support',                   
'department_temp',                      
'new']


In [33]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
import numpy as np
###initialize Boruta

X2=data[feature_cols].values

y= ((data['resposta']).values)

forest = RandomForestRegressor(
   n_jobs = -1, 
   max_depth = 3
)
boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 100 # number of trials to perform
)
### fit Boruta (it accepts np.array, not pd.DataFrame)
boruta.fit(X2,y)
### print results
green_area = data[feature_cols].columns[boruta.support_].to_list()
blue_area = data[feature_cols].columns[boruta.support_weak_].to_list()
print('features in the green area:', green_area)
print('features in the blue area:', blue_area)
#

features in the green area: ['avg_monthly_hrs', 'last_evaluation', 'n_projects', 'satisfaction']
features in the blue area: []


In [17]:
#logistica 
from sklearn.model_selection import train_test_split
feature_cols = [
'avg_monthly_hrs', 'last_evaluation', 'n_projects', 'satisfaction'
]
X=data[feature_cols]

y=data['resposta']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

In [19]:
from sklearn.linear_model import LogisticRegressionCV
from xgboost import XGBClassifier


model = XGBClassifier(max_depth=3,n_estimators=20)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)


[15:22:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\fgeba\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



In [20]:
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))


print('R^2 Training Score: {:.2f}  \nR^2 Validation Score: {:.2f}'.format(model.score(X_train, y_train), 
                                                                                        
                                                                                             model.score(X_test, y_test)))



[[2160   40]
 [ 119  531]]
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      2200
           1       0.93      0.82      0.87       650

    accuracy                           0.94      2850
   macro avg       0.94      0.90      0.92      2850
weighted avg       0.94      0.94      0.94      2850

0.9442105263157895
R^2 Training Score: 0.94  
R^2 Validation Score: 0.94


In [21]:
def ks(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
 #   data=data.drop(['target0','bucket'],axis=1)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    maxi= max(kstable['KS'])
    #Display KS
   # from colorama import Fore
    #print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return( maxi )

In [22]:


new_y=model.predict_proba(X_train)

df = pd.DataFrame(new_y,columns=['good', 'bad'])


df2=df.reset_index(drop=True).merge(y_train.reset_index(drop=True), left_index=True, right_index=True)
df3=df2.reset_index(drop=True).merge(X_train.reset_index(drop=True), left_index=True, right_index=True)

mydf =ks(data=df3,target="resposta", prob="bad")

        min_prob  max_prob  events  nonevents event_rate nonevent_rate  \
Decile                                                                   
1       0.933425  0.996744     424          0     15.46%         0.00%   
2       0.590759  0.929501    1684        154     61.42%         1.78%   
3       0.184364  0.582151     418        717     15.24%         8.28%   
4       0.082108  0.182539      81       1082      2.95%        12.50%   
5       0.049464  0.081706      39       1018      1.42%        11.76%   
6       0.038272  0.049393      25       1024      0.91%        11.83%   
7       0.028460  0.038198      28       1255      1.02%        14.50%   
8       0.022139  0.028435      27       1088      0.98%        12.57%   
9       0.017661  0.022075       8       1132      0.29%        13.08%   
10      0.005292  0.017606       8       1187      0.29%        13.71%   

       cum_eventrate cum_noneventrate    KS  
Decile                                       
1             15.46

In [23]:
!pip install -q gradio 

In [24]:
import gradio as gr
import pandas as pd
import json

In [25]:
def predict_survival(avg_monthly_hrs,last_evaluation, n_projects, satisfaction):
    df = pd.DataFrame.from_dict({'avg_monthly_hrs': [avg_monthly_hrs],'last_evaluation': [last_evaluation],
                                 'n_projects': [n_projects],'satisfaction': [satisfaction]
                                })
    pred = model.predict_proba(df)[0]
    return {'Permanece na empresa': float(pred[0]), 'Sai da empersa': float(pred[1])}

In [26]:
print(predict_survival(180,0.2,3,0.2))

{'Permanece na empresa': 0.9718173742294312, 'Sai da empersa': 0.02818259969353676}


In [27]:

avg_monthly_hrs = gr.inputs.Slider(minimum=49, maximum=310, default=180, label="Horas trabalhadas")
last_evaluation =gr.inputs.Slider(minimum=0, maximum=1, default=0.5, label="Última Avaliação")
n_projects = gr.inputs.Radio([ 1, 2,3,4,5,6], label="Número de projetos")
satisfaction = gr.inputs.Slider(minimum=0, maximum=1, default=0.5, label="Satisfação")

gr.Interface(predict_survival, [avg_monthly_hrs,
                                last_evaluation, n_projects,satisfaction
                               ], "label", live=True).launch(share=True);


Running locally at: http://127.0.0.1:7860/
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/hosted (NEW!)
Running on External URL: https://39146.gradio.app
Interface loading below...


In [122]:
pip install explainerdashboard --user

  Using cached explainerdashboard-0.3.0.1-py3-none-any.whl (268 kB)
  Using cached shortuuid-1.0.1-py3-none-any.whl (7.5 kB)
Processing c:\users\fgeba\appdata\local\pip\cache\wheels\85\e9\f4\033d9bd19fe04ba8b3b41b6cfaec3c4634e85bdcdd82858ff3\dash-1.19.0-py3-none-any.whl
  Using cached shap-0.38.1-cp37-cp37m-win_amd64.whl (413 kB)
  Using cached flask_simplelogin-0.0.7-py2.py3-none-any.whl (18 kB)
Processing c:\users\fgeba\appdata\local\pip\cache\wheels\59\bb\83\6b7c816105cc26e388a8166f43447fc26d350ee3b808209ea8\dtreeviz-1.1.4-py3-none-any.whl
  Using cached dash_bootstrap_components-0.11.1-py2.py3-none-any.whl (187 kB)
  Using cached jupyter_dash-0.4.0-py3-none-any.whl (20 kB)
Processing c:\users\fgeba\appdata\local\pip\cache\wheels\19\b2\02\3c3f05988ff92f02c52ce4e081859d423537e8e9b13f673c02\dash_auth-1.4.1-py3-none-any.whl
Processing c:\users\fgeba\appdata\local\pip\cache\wheels\02\a5\de\374685270e68a4a49867f9c0ddd477a85e09eb1a2cb0998f92\dash_core_components-1.15.0-py3-none-any.whl
Pr

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [28]:
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
explainer = ClassifierExplainer(model, X_test, y_test)

Note: shap=='guess' so guessing for XGBClassifier shap='tree'...
Detected XGBClassifier model: Changing class type to XGBClassifierExplainer...
Note: model_output=='probability'. For XGBClassifier shap values normally get calculated against X_background, but paramater X_background=None, so using X instead
Generating self.shap_explainer = shap.TreeExplainer(model, X, model_output='probability', feature_perturbation='interventional')...
Note: Shap interaction values will not be available. If shap values in probability space are not necessary you can pass model_output='logodds' to get shap values in logodds without the need for a background dataset and also working shap interaction values...


In [ ]:
ExplainerDashboard(explainer).run()

Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
Generating layout...
Calculating shap values...
Generating xgboost model dump...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating pred_percentiles...
Calculating prediction probabilities...
Calculating predictions...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://localhost:8050
Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

